In [1]:
# Importando Bibliotecas:
import pandas as pd
import numpy as np
from pathlib import Path

# Cálculo do Tempo médio:
def calcular_tempo_medio(caminho_arquivo_entrada: str, pasta_saida: str):
    # 1. Ler o arquivo de entrada
    caminho_entrada = Path(caminho_arquivo_entrada)
    if not caminho_entrada.is_file():
        print(f"Error: Arquivo de entrada não encontrado em '{caminho_entrada}'")
        return
    
    # 2. Criar pasta de saída se não existir:
    caminho_saida = Path(pasta_saida)
    caminho_saida.mkdir(parents=True, exist_ok=True)
    print(f"Resultados serão salvos na pasta: '{caminho_saida}'")

    # 3. Carregamento dos Dados:
    try:
        df = pd.read_csv(caminho_entrada, sep=',', encoding='utf-8', low_memory=False)
        
        # 4. Limpeza e Preparação dos Dados:
        # 4.1. Converte as colunas de data para o formato datetime:
        df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce', format='%Y-%m-%d')
        df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce', format='%Y-%m-%d')

        # 4.2. Remove linhas com datas nulas:
        df = df.dropna(subset=['data_distribuicao', 'data_baixa'])

        # 4.3. Calcula a diferença de tempo (duração):
        df['tempo_processual'] = (df['data_baixa'] - df['data_distribuicao']).dt.days

        # 4.4. Garante que não há durações negativas:
        df = df[df['tempo_processual'] >= 0]
        
        # 5. Calcula o Tempo Médio por Instância, Entrância, Comarca e Serventia:
        media_por_inst_ent_com_ser = df.groupby(['instancia','entrancia','comarca', 'serventia'])['tempo_processual'].mean().round(2).reset_index()
        media_por_inst_ent_com_ser.rename(columns={'tempo_processual': 'tempo_medio_dias'}, inplace=True)

        # 5. Salvando os Resultados em CSV:
        caminho_inst_ent_com_ser = caminho_saida / 'media_por_inst_ent_com_ser.csv' 
        media_por_inst_ent_com_ser.to_csv(caminho_inst_ent_com_ser, index=False, sep=',', encoding='utf-8')

        print("Cálculo do tempo médio concluído com sucesso!")
   
    except FileNotFoundError:
        print(f"Error: O arquivo '{caminho_arquivo_entrada}' não foi encontrado.")
    except KeyError as e:
        print(f"Error: Coluna ausente no DataFrame - {e}. Verifique se os nomes das colunas estão corretos.")
    except Exception as e:
        print(f"Error: Ocorreu um erro inesperado durante o processamento: {e}")

# Aplicando a função:
caminho_arquivo_entrada = 'uploads/relatorio_processos_2024-07-01_a_2025-06-30.csv'
pasta_saida = 'resultados_da_analise'

calcular_tempo_medio(caminho_arquivo_entrada, pasta_saida)


Resultados serão salvos na pasta: 'resultados_da_analise'
Cálculo do tempo médio concluído com sucesso!
